### Testing Temp -> Demand conversion ###

Based on Hannah Bloomfield's S2S4E scripts, converting the NUTS0 aggregated temperature data into national demand (for ERA5 first).

Before moving onto explore how we might go about doing this conversion for the decadal predictions at daily timescales (e.g HadGEM3-GC31-MM in the first instance).

In [ ]:
# cell magic
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Load local modules
import os
import sys
import glob
import importlib

# Import external modules
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
! pip install regionmask

In [ ]:
! pip install ncdata

In [ ]:
# Import the functions
sys.path.append("/home/users/benhutch/unseen_functions")

import functions_demand as func_dm

import bias_functions as bias_func

import load_wind_functions as lw_func

In [ ]:
# import dictionaries
sys.path.append("/home/users/benhutch/unseen_functions")

import unseen_dictionaries as udicts

In [ ]:
# set up the global variables
# set up for a single country
first_year = 1960
last_year = 1960
first_month = 1
last_month = 1
init_years = np.arange(1960, 2018 + 1, 1)
variable = "tas"
lead_time = 1
obs_variable = "t2m"
country = "United Kingdom"
country_name = "United_Kingdom"

# set up the path to the observations (daily, temperature)
obs_path = "/home/users/benhutch/ERA5/ERA5_t2m_daily_1950_2020.nc"

# set up the path to the model data (daily, temperature)
model_path = "/work/scratch-nopw2/benhutch/test_nc/tas_bias_correction_HadGEM3-GC31-MM_lead1_month11_init1960-1960.nc"

# model path month 2
model_path2 = "/work/scratch-nopw2/benhutch/test_nc/tas_bias_correction_HadGEM3-GC31-MM_lead1_month12_init1960-1960.nc"

# Test ds
test_file_path = "/badc/cmip6/data/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1960-r1i1p1f2/day/tas/gn/files/d20200417/tas_day_HadGEM3-GC31-MM_dcppA-hindcast_s1960-r1i1p1f2_gn_19601101-19601230.nc"

In [ ]:
# %%time

# # Load the file
# model_ds = xr.open_dataset(test_file_path)

# # Load the observations
# # But first regrid
# obs = bias_func.load_and_rg_obs(
#     model_ds=model_ds,
#     obs_variable=obs_variable,
#     obs_path=obs_path,
#     init_years=init_years,
#     lead_time=lead_time,
#     rg_algo="bilinear",
#     grid_bounds=[-180.0, 180.0, -90.0, 90.0],
#     periodic=True,
#     parallel=False,
# )

# # Select the gridbox
# obs = bias_func.select_gridbox(
#     ds=obs,
#     grid=udicts.eu_grid_constrained,
#     calc_mean=False,
# )

In [ ]:
# # Check out the observations
# obs

In [ ]:
importlib.reload(lw_func)

In [ ]:
# # Apply the country mask to the obs data
# obs = lw_func.apply_country_mask(
#     ds=obs,
#     country=country,
#     lon_name="lon",
#     lat_name="lat",
# )

In [ ]:
importlib.reload(func_dm)

In [ ]:
# # Calculate the mean for the country
# obs = func_dm.calc_spatial_mean(
#     ds=obs,
#     country=country_name,
#     variable=obs_variable,
#     convert_kelv_to_cel=True,
# )
    

In [ ]:
# # Calculate HDD and CDD
# obs_df = func_dm.calc_hdd_cdd(
#     df=obs,
#     temp_suffix=obs_variable,
# )

In [ ]:
# # Calculate the national wd_demand
# obs_df = func_dm.calc_national_wd_demand(
#     df=obs_df,
# )

In [ ]:
# obs_df

Observations successfully processed, can we do the same for the model data?

In [ ]:
importlib.reload(lw_func)

In [ ]:
# loop over the monhts
months_list = [11, 12]

# set up the base_dir
base_dir = "/work/scratch-nopw2/benhutch/test_nc/"

variable_saved = "__xarray_dataarray_variable__"

# Set up an empty dataframe
combined_df = pd.DataFrame()

# loop over the months
for month_idx in tqdm(months_list):
    # Set up the fname
    fname_month = f"tas_bias_correction_HadGEM3-GC31-MM_lead1_month{month_idx}_init1960-1960.nc"

    # Set up the path
    path_month = os.path.join(base_dir, fname_month)

    # Apply the country mask
    model_month = lw_func.apply_country_mask(
        ds=xr.open_dataset(path_month),
        country=country,
        lon_name="lon",
        lat_name="lat",
    )

    # Calculate the mean for the country
    model_month = func_dm.calc_spatial_mean(
        ds=model_month,
        country=country_name,
        variable=variable_saved,
        variable_name=variable,
        convert_kelv_to_cel=True,
    )

    # combine the dataframes
    combined_df = pd.concat([combined_df, model_month], ignore_index=True)

In [ ]:
combined_df

In [ ]:
# Calculate the heating degree days and cooling degree days
model_df = func_dm.calc_hdd_cdd(
    df=combined_df,
    country_name=country_name,
    variable_name=variable,
)

In [ ]:
# Calculate the weather dependent demand
model_df = func_dm.calc_national_wd_demand(
    df=model_df,
    country_name=country_name,
)

In [ ]:
model_df

In [ ]:
# set up the fname for the fike
fname = f"wd_demand_{country_name}_month11-12_init1960-1960.csv"

# save the df
func_dm.save_df(
    df=model_df,
    fname=fname,
)

In [ ]:
# Test loading the file
dir = "/gws/nopw/j04/canari/users/benhutch/met_to_energy_dfs"

fname = "wd_demand_United_Kingdom_month11-12_init1960-1960.csv.csv"

# Load the file
df = pd.read_csv(os.path.join(dir, fname))

In [ ]:
df

In [ ]:
# Apply the country mask to the model data
model = lw_func.apply_country_mask(
    ds=model,
    country=country,
    lon_name="lon",
    lat_name="lat",
)

model_2 = lw_func.apply_country_mask(
    ds=model_2,
    country=country,
    lon_name="lon",
    lat_name="lat",
)

In [ ]:
importlib.reload(func_dm)

In [ ]:
# Calculate the spatial mean for the country
model_df = func_dm.calc_spatial_mean(
    ds=model,
    country=country_name,
    variable="__xarray_dataarray_variable__",
    variable_name=variable,
    convert_kelv_to_cel=True,
)

model_df_2 = func_dm.calc_spatial_mean(
    ds=model_2,
    country=country_name,
    variable="__xarray_dataarray_variable__",
    variable_name=variable,
    convert_kelv_to_cel=True,
)

In [ ]:
model_df

In [ ]:
model_df_2

In [ ]:
combined_model_df = pd.concat([model_df, model_df_2], ignore_index=True)

In [ ]:
combined_model_df

In [ ]:
# Calculate the heating degree days and cooling degree days
model_df = func_dm.calc_hdd_cdd(
    df=combined_model_df,
    country_name=country_name,
    variable_name=variable,
)

In [ ]:
model_df

In [ ]:
importlib.reload(func_dm)

In [ ]:
# Calculate the weather dependent demand
model_df = func_dm.calc_national_wd_demand(
    df=model_df,
    country_name=country_name,
)

In [ ]:
model_df

In [ ]:
# plot the UK demand
# with lead on the x-axis
# United_Kingdom_demand on the y-axis
# and plot all of the members
fig, ax = plt.subplots(figsize=(10, 5))

# extract hthe unique members
unique_members = model_df.member.unique()


model_df_grouped = model_df.groupby("lead")

# include the 5-95% range
ax.fill_between(
    model_df_grouped.mean().index,
    model_df_grouped.quantile(0.05).United_Kingdom_demand,
    model_df_grouped.quantile(0.95).United_Kingdom_demand,
    alpha=0.2,
    color="red",
)


for member in unique_members:
    temp_df = model_df[model_df.member == member]
    ax.plot(temp_df.lead, temp_df.United_Kingdom_demand, label=member)


# include an ensemble mean
ensemble_mean = model_df.groupby("lead").mean()

ax.plot(ensemble_mean.index, ensemble_mean.United_Kingdom_demand, label="Ensemble Mean", color="red", linestyle="--")


# # subset the obs to the month 11 and 12
obs_df_month = obs_df[obs_df.index.month.isin([11, 12])]

# set up a new axis for lead (the time axis as 1 to len of the obs_df_month)
obs_df_month["lead"] = np.arange(1, len(obs_df_month) + 1)

# plot the obs
ax.plot(obs_df_month.lead, obs_df_month.United_Kingdom_demand, label="Obs", color="black")

# include a legend
ax.legend()